<a href="https://colab.research.google.com/github/duraichellam001-tech/EV-PhysicsCalib-Model/blob/main/notebooks/03_model_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F

# ---- Physical constants ----
g = 9.81
rho = 1.225


class PhysicsGuidedEVModel(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=64):
        """
        input_dim: number of features (v, a, etc.)
        hidden_dim: size of LSTM hidden layer
        """
        super().__init__()

        # ---- Encoder for time-series ----
        self.encoder = nn.LSTM(input_dim, hidden_dim, batch_first=True, num_layers=1)

        # ---- Parameter regression head ----
        self.param_head = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 4),  # Crr, CdA, eta, Paux
            nn.Sigmoid()
        )

        # ---- Nominal parameter scaling ----
        self.param_bounds = {
            'Crr': (0.007, 0.018),
            'CdA': (0.6, 1.0),
            'eta': (0.85, 0.98),
            'Paux': (200.0, 1500.0),
        }

    def scale_params(self, raw):
        """Map 0–1 sigmoid outputs → physical ranges"""
        Crr = self.param_bounds['Crr'][0] + (self.param_bounds['Crr'][1] - self.param_bounds['Crr'][0]) * raw[:, 0]
        CdA = self.param_bounds['CdA'][0] + (self.param_bounds['CdA'][1] - self.param_bounds['CdA'][0]) * raw[:, 1]
        eta = self.param_bounds['eta'][0] + (self.param_bounds['eta'][1] - self.param_bounds['eta'][0]) * raw[:, 2]
        Paux = self.param_bounds['Paux'][0] + (self.param_bounds['Paux'][1] - self.param_bounds['Paux'][0]) * raw[:, 3]
        return Crr, CdA, eta, Paux

    def forward(self, x, mass_kg):
        """
        x: [batch, seq_len, input_dim] (v, a)
        mass_kg: tensor [batch] (known per sample)
        """
        batch_size, seq_len, _ = x.shape
        v = x[:, :, 0]
        a = x[:, :, 1]

        # ---- Encode time-series ----
        _, (h_last, _) = self.encoder(x)
        h_last = h_last[-1]  # [batch, hidden_dim]

        # ---- Predict physical parameters ----
        raw_params = self.param_head(h_last)
        Crr, CdA, eta, Paux = self.scale_params(raw_params)

        # ---- Physics layer ----
        v2 = v ** 2
        F_rr = mass_kg.view(-1, 1) * g * Crr.view(-1, 1)
        F_aero = 0.5 * rho * CdA.view(-1, 1) * v2
        F_acc = mass_kg.view(-1, 1) * a
        F_total = F_rr + F_aero + F_acc
        P_pred = (F_total * v) / (eta.view(-1, 1) + 1e-9) + Paux.view(-1, 1)

        return P_pred, {'Crr': Crr, 'CdA': CdA, 'eta': eta, 'Paux': Paux}


In [3]:
import torch

model = PhysicsGuidedEVModel()
x = torch.randn(1, 600, 2)   # random 1 drive cycle, 600 samples, (v, a)
mass = torch.tensor([1500.0])

P_pred, params = model(x, mass)
print("Power output shape:", P_pred.shape)
print("Predicted parameters:")
for k, v in params.items():
    print(f"  {k}: {v.item():.4f}")


Power output shape: torch.Size([1, 600])
Predicted parameters:
  Crr: 0.0124
  CdA: 0.8089
  eta: 0.9159
  Paux: 874.4319
